In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neural_network import MLPRegressor

import SQLconnection
import database
import numpy as np

data = database.getDataAndPriceFeaturesAndRealizedVarianceData ()


def formatting ():
    listOfDates = data['date_from_1996'].tolist ()
    distinctListOfDates = []
    for d in listOfDates:
        if d not in distinctListOfDates:
            distinctListOfDates.append (d)

    mapOfdatesAndOthers = {}
    for date in distinctListOfDates:
        mapOfdatesAndOthers[date] = (SQLconnection.something (date=date)['priceFeatures'].tolist (),
                                     SQLconnection.somethingelse (date=date)['futureRealizedVariance'].tolist ())

    return mapOfdatesAndOthers


def additionalFormat ():
    my_map = formatting ()
    keys = my_map.keys ()
    X = []
    y = []
    for item in keys:
        X.append (my_map[item][0])
        y.append (my_map[item][1])
    y = np.array (y)
    X = np.array (X)
    return X, y

def splittingDataAndTrain ():
    X, y = additionalFormat ()
    time_series_split = TimeSeriesSplit (n_splits=300)
    for train_index, test_index in time_series_split.split (X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        regression = MLPRegressor (hidden_layer_sizes=80).fit (X_train, y_train)
        regression.predict (X_test)
        regression.score (X_test, y_test)

splittingDataAndTrain()

            id  date_from_1996  priceFeatures  futureRealizedVariance
0            2               3   8.675390e-07                     NaN
1            3               3   1.363280e-06                     NaN
2            4               3   1.487210e-06                     NaN
3            5               3   1.611140e-06                     NaN
4            6               3   1.735080e-06                     NaN
...        ...             ...            ...                     ...
304513  304515            9131   3.763590e-06                0.006439
304514  304516            9131   3.660750e-06                0.006439
304515  304517            9131   3.593370e-06                0.006439
304516  304518            9131   3.567370e-06                0.006439
304517  304519            9131   3.572100e-06                0.006439

[304518 rows x 4 columns]


In [ ]:
splittingDataAndTrain()